<a href="https://colab.research.google.com/github/Lethokuhle222/Lethokuhle222/blob/main/Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install --upgrade transformers torch --quiet


In [66]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import files
uploaded = files.upload()




Saving IMDB Dataset.csv to IMDB Dataset (1).csv


In [4]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.shape

(50000, 2)

In [7]:
type(data)

pandas.core.frame.DataFrame

In [8]:
data.tail()


,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
data["sentiment"].value_counts()


,count
sentiment,
positive,25000
negative,25000


In [10]:
# one hot encoding
# label encoder

In [11]:
# positive -> 1
# negative -> 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [12]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
data.tail()


,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [14]:
data["sentiment"].value_counts()


,count
sentiment,
1,25000
0,25000


In [15]:
# LSTM -> LONG SHORT TERM MEMORY
# RNN -> TEXTUAL DATA

In [16]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)


In [18]:
train_data.shape


(40000, 2)

In [19]:
test_data.shape


(10000, 2)

In [20]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [21]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [22]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [23]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [24]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [25]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [26]:
# LSTM MODEL BUILDING

In [27]:
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [28]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])


In [30]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 292s 575ms/step - accuracy: 0.7162 - loss: 0.5418 - val_accuracy: 0.8470 - val_loss: 0.3638
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 287s 573ms/step - accuracy: 0.8562 - loss: 0.3456 - val_accuracy: 0.8425 - val_loss: 0.3720
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 322s 574ms/step - accuracy: 0.8822 - loss: 0.2970 - val_accuracy: 0.8709 - val_loss: 0.3364
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 330s 589ms/step - accuracy: 0.8984 - loss: 0.2551 - val_accuracy: 0.8547 - val_loss: 0.3673
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 320s 586ms/step - accuracy: 0.8930 - loss: 0.2715 - val_accuracy: 0.8729 - val_loss: 0.3297


In [31]:
model.save("model.h5")


In [32]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [33]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 82ms/step - accuracy: 0.8722 - loss: 0.3152


In [34]:
print(loss)


0.3098572790622711


In [35]:
print(accuracy)


0.8766000270843506


In [36]:
# Building Predictive System


In [37]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [38]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step


'positive'

In [39]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


'positive'

In [40]:
predictive_system("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


'negative'

In [41]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


'positive'

In [42]:
predictive_system("Pretty boring and story unclear")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


'negative'

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

In [44]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(train_data["review"])
X_test_vec  = vectorizer.transform(test_data["review"])

In [45]:
# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, Y_train)
nb_preds = nb_model.predict(X_test_vec)
print("Naive Bayes Accuracy:", accuracy_score(Y_test, nb_preds))
print(classification_report(Y_test, nb_preds))

Naive Bayes Accuracy: 0.8508
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      4961
           1       0.85      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [46]:
# Support Vector Classifier (Linear Kernel)
svc_model = LinearSVC()
svc_model.fit(X_train_vec, Y_train)
svc_preds = svc_model.predict(X_test_vec)
print("SVC Accuracy:", accuracy_score(Y_test, svc_preds))
print(classification_report(Y_test, svc_preds))

SVC Accuracy: 0.8772
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      4961
           1       0.87      0.89      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



DistilBERT (Transformer Model)

In [47]:
!pip install transformers datasets torch --quiet

In [48]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset


In [49]:
tokenizer_bert = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [50]:
# Encode data
train_encodings = tokenizer_bert(list(train_data["review"]), truncation=True, padding=True, max_length=200)
test_encodings  = tokenizer_bert(list(test_data["review"]), truncation=True, padding=True, max_length=200)

In [51]:
# Torch Dataset
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [52]:
train_dataset = IMDbDataset(train_encodings, Y_train)
test_dataset  = IMDbDataset(test_encodings, Y_test)

In [53]:
# Model
bert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
!pip install --upgrade transformers


In [55]:
# upgrade transformers + torch (recommended)
!pip install -q --upgrade transformers torch


In [56]:
import transformers
from transformers import TrainingArguments

In [57]:
print("transformers version:", transformers.__version__)

def make_training_args():
    base_kwargs = dict(
        output_dir='./results',
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
    )

    # Try modern API first
    try:
        return TrainingArguments(**{**base_kwargs, "evaluation_strategy": "epoch"})
    except TypeError:
        # Older transformers used evaluate_during_training
        try:
            return TrainingArguments(**{**base_kwargs, "evaluate_during_training": True})
        except TypeError:
            # Last resort: construct with base args only and warn user
            print("Warning: Your transformers version does not accept 'evaluation_strategy' nor 'evaluate_during_training'.")
            print("You should upgrade transformers for full Trainer functionality:")
            print("  !pip install --upgrade transformers torch")
            return TrainingArguments(**base_kwargs)

training_args = make_training_args()
print(training_args)

transformers version: 4.56.1
You should upgrade transformers for full Trainer functionality:
  !pip install --upgrade transformers torch
TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_dela

In [58]:
# Define compute_metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [59]:
import torch
from torch.utils.data import Dataset

class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


In [60]:
from transformers import DistilBertTokenizerFast

tokenizer_bert = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer_bert(list(train_data["review"]), truncation=True, padding=True, max_length=200)
test_encodings  = tokenizer_bert(list(test_data["review"]), truncation=True, padding=True, max_length=200)


In [61]:
train_dataset = IMDbDataset(train_encodings, Y_train)
test_dataset  = IMDbDataset(test_encodings, Y_test)


In [62]:
!pip install --upgrade transformers torch


In [64]:
!pip install --upgrade transformers torch --quiet


In [72]:


# Trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [1]:
# Train
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
# Evaluate
results = trainer.evaluate()
print("DistilBERT Accuracy:", results["eval_accuracy"])

In [ ]:
results = {
    "LSTM": model.evaluate(X_test, Y_test, verbose=0)[1],
    "Naive Bayes": accuracy_score(Y_test, nb_preds),
    "SVC": accuracy_score(Y_test, svc_preds),
    "DistilBERT": bert_results[1]
}

print("\nModel Performance Comparison:")
for m, acc in results.items():
    print(f"{m}: {acc:.4f}")
